In [41]:
import pandas as pd
import requests as r
import time

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC

In [42]:
PATH=ChromeDriverManager().install()   #'driver/chromedriver'
url='https://www.gog.com/promo/black_friday_2021_rpg'

driver=webdriver.Chrome(PATH)
driver.get(url)
time.sleep(3)
driver.find_element_by_xpath('//*[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]').click()
html=driver.page_source
driver.close()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/jmc/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
<ipython-input-42-f5c09198c7ef>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)
<ipython-input-42-f5c09198c7ef>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]').click()


In [43]:
soup=bs(html, 'html.parser')
product_names= [name.text.split(' - ')[0] for name in soup.find_all('span', class_='product-title__text')]
product_category= [cat.text for cat in soup.find_all('span', class_='product-row__genre')]
release_year= [y.text for y in soup.find_all('span', class_='product-row__date')]
original_price= [op.text for op in soup.find_all('s', class_='product-row-price--old')]
disc_price= [dp.text for dp in soup.find_all('b', class_='product-row-price--new')]

In [44]:
discounts_df= pd.DataFrame({'game': product_names,
                            'cat': product_category,
                            'r_year': release_year,
                            'price': original_price,
                            'disc_price': disc_price})

discounts_df['r_year'] = discounts_df['r_year'].astype('int')
discounts_df['price'] = discounts_df['price'].astype('float')
discounts_df['disc_price'] = discounts_df['disc_price'].astype('float')
discounts_df['pct_discounted'] = round((discounts_df['price']-discounts_df['disc_price'])/discounts_df['price']*100,0).astype('int')

In [68]:
discounts_df['game'].iloc[1] = discounts_df['game'].iloc[1].replace('2', 'II')

In [75]:
url_games = []

for game in discounts_df['game']:
    url_games.append(f"https://www.metacritic.com/game/pc/{game.replace(':', '').replace(' - ', '-').replace(' ', '-').lower()}")

In [112]:
user_scores=[]
user_ratings=[]

for game in url_games:
    try:
        url=game
        html=r.get(url, headers=headers).text
        soup=bs(html, 'html.parser')
        user_score=soup.find('div', class_='userscore_wrap feature_userscore')
        user_scores.append(user_score.find_all('div')[1].text)
        user_ratings.append(int(user_score.find('span', class_='count').find('a').text.split(' ')[0]))
    except:
        user_scores.append(-1)
        user_ratings.append(-1)

In [113]:
discounts_df['user_score']=user_scores
discounts_df['review_number']=user_ratings
discounts_df.head()